# Joins

In [1]:
import pandas as pd

So far he have only worked on a single table, although, our database contains two tables:
* Canvas (parent)
* Graffiti (child)

Using database lingo, we say that `canvas` and `graffiti` have a **parent-child** type of relationship.

In any database table, as the one we have, each row needs to be **identified uniquely**. This identifier is usually refer to as a **primary key**. In our case, the **primary key** in the `canvas` table is the column called '*id*'. The **primary key** in the `graffiti` table is also called '*id*'. However, if you notice carefully, there is another key included in the `graffiti` table called '*canvas_id*' which corresponds to the '*id*' (= *primary key*) in the `canvas` table. This '*id*' is known as a **foreign key**. Essentially, a *foreign key* is the primary key from another table.

If you recall, in order to record a graffito in the `graffiti` table, we first need to have an entry in the `canvas` table. Once we have an entry in the `canvas` table, we can start recording graffiti. The relationship between the `canvas` table and the `graffiti` table is said to be a *one-to-many* relationship (see below). This means that one entry in the `canvas` table can refer to at least one but potentially many entries in the `graffiti` table.

In general, the way two tables in a database relate to each other can be of three different types:     
- *one-to-one*: one row(record) in table A corresponds to only one row(record) in table B.
- *one-to-many*: one row(record) in table A can correspond to many rows(records) in table B.
- *many-to-many*: many rows(records) in table A can correpond to many rows(records) in table B.

If you think about it, the fact that the relationship between the Canvas and the `graffiti` table is *one-to-many* makes sense. The `canvas` table really records the object or surface onto which one or several graffiti can be written. By now you are probably aware of the following:
* `canvas` table is used to collect information about:
  * Nature of the object or surface onto which a graffiti is written
  * Graffiti location
  * Level of visibility of the graffiti's location
  * Level of surveillance of the graffiti's location
  * Level of accesibility of the graffiti's location
* `graffiti` table is used to collect information about:
  * Graffiti Typw
  * Nature of message
  * Characteristics of the graffiti
  * Relationship with other graffiti

The result of recording different types of information into different tables means that if we want to ask some question that requires information from the different tables, e.g. what is the relationship between the level of surveillance and type of graffity? we really need to connect the two tables. This is accomplish doing a **join** (or a **merge**).

Pandas has two functions to create joins: [`.join()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html) and [`.merge()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html). Here we will use `.merge()` primarily.

Before we do some simple queries with the graffiti data, let's use a very simple DataFrame to illustrate different types of joins. This database is made out of two tables: students and courses.

In [2]:
id = [1,2,3,4,5,6]
Name = ['Joe', 'Maria', 'Zoe', 'Matt', 'Celeste', 'Franco']
Course_id = [3, 2, 2, 1, 3, 5]
students = pd.DataFrame(zip(id, Name, Course_id), columns=['Id', 'Name', 'Course_id'])
students

,Id,Name,Course_id
0,1,Joe,3
1,2,Maria,2
2,3,Zoe,2
3,4,Matt,1
4,5,Celeste,3
5,6,Franco,5


In [3]:
Course = ['Intro to Harry Potter', 'Intermediate Stats',  'Graffiti Analysis', 'Film Studies']
Id = list(range(1,5))
Prereqs = ['No', 'Yes', 'Yes', 'No']
courses = pd.DataFrame(zip(Id, Course, Prereqs), columns = ['Id', 'Course', 'Prerequisites'])
courses

,Id,Course,Prerequisites
0,1,Intro to Harry Potter,No
1,2,Intermediate Stats,Yes
2,3,Graffiti Analysis,Yes
3,4,Film Studies,No


### Types of joins

Pandas has two functions to create joins: [`.join()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html) and [`.merge()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html). Here we will use `.merge()` primarily.

There are several types of [joins](https://www.analyticsvidhya.com/blog/2020/02/joins-in-pandas-master-the-different-types-of-joins-in-python/). Here are the most commons ones,    

```{image} ../../images/joins.png
:width: 55%
:align: center
```

When joining two tables, we will often need to make referece to some common piece of information that tables share (i.e. to one or several columns). In the Graffiti databse case, the only bit of information that both tables, `canvas` and `graffito`, share is the 'Canvas id'. While the name is different in each table ('id' in the `canvas` table and 'canvas_id' in the `graffito` table) they are both making reference to the same information. They are both referring to the *primary key* in the `canvas` table. 

Let's try to understand the differences between these different types of joins using our simple database: 

* **Inner Join**: Returns only those rows in both the student and the course  tables wherenever the 'Course_Id' in the `students` table coincides with 'Id' in the `courses` table.

* **Left (Outer) Join**: Returns all rows in the `students` table regardless of whether they match some row in the `Courses`table.

* **Right (Outer) Join**: Returns all rows in the `courses` table regardless of whether they match a row in the `students` table.

* **Full (Outer) Join**: Returns all rows in the `students` and `courses` tables *regardless* of whether the rows in one table matches any row in the other table .

### Inner Join

Returns only those rows in both the student and the course  tables wherenever the 'Course_Id' in the `students` table coincides with 'Id' in the `courses` table.

In [4]:
inner = pd.merge(students, courses, left_on= 'Course_id', right_on='Id', how='inner')
inner

,Id_x,Name,Course_id,Id_y,Course,Prerequisites
0,1,Joe,3,3,Graffiti Analysis,Yes
1,2,Maria,2,2,Intermediate Stats,Yes
2,3,Zoe,2,2,Intermediate Stats,Yes
3,4,Matt,1,1,Intro to Harry Potter,No
4,5,Celeste,3,3,Graffiti Analysis,Yes


In [5]:
# reverse join
inner = pd.merge(courses, students, left_on= 'Id', right_on='Course_id', how='inner')
inner

,Id_x,Course,Prerequisites,Id_y,Name,Course_id
0,1,Intro to Harry Potter,No,4,Matt,1
1,2,Intermediate Stats,Yes,2,Maria,2
2,2,Intermediate Stats,Yes,3,Zoe,2
3,3,Graffiti Analysis,Yes,1,Joe,3
4,3,Graffiti Analysis,Yes,5,Celeste,3


In [6]:
# Using merge directly with a dataframe
students.merge(courses, left_on= 'Course_id', right_on='Id', how='inner', suffixes=(None,'_crse' ))

,Id,Name,Course_id,Id_crse,Course,Prerequisites
0,1,Joe,3,3,Graffiti Analysis,Yes
1,2,Maria,2,2,Intermediate Stats,Yes
2,3,Zoe,2,2,Intermediate Stats,Yes
3,4,Matt,1,1,Intro to Harry Potter,No
4,5,Celeste,3,3,Graffiti Analysis,Yes


### Left Outer Join (Left Join) 

Returns all rows in the `students` table regardless of whether they match some row in the `Courses`table

In [7]:
# left join
left = pd.merge(students, courses, left_on= 'Course_id', right_on='Id', how='left')
left

,Id_x,Name,Course_id,Id_y,Course,Prerequisites
0,1,Joe,3,3.0,Graffiti Analysis,Yes
1,2,Maria,2,2.0,Intermediate Stats,Yes
2,3,Zoe,2,2.0,Intermediate Stats,Yes
3,4,Matt,1,1.0,Intro to Harry Potter,No
4,5,Celeste,3,3.0,Graffiti Analysis,Yes
5,6,Franco,5,NaN,NaN,NaN


Notice that the row with student *Franco* does not match any record in the `Courses` table

### Right Outer Join (Right Join) 

Returns all rows in the `courses` table regardless of whether they match a row in the `students` table

In [8]:
# right join
right = pd.merge(students, courses, left_on= 'Course_id', right_on='Id', how='right')
right.head(15)

,Id_x,Name,Course_id,Id_y,Course,Prerequisites
0,4.0,Matt,1.0,1,Intro to Harry Potter,No
1,2.0,Maria,2.0,2,Intermediate Stats,Yes
2,3.0,Zoe,2.0,2,Intermediate Stats,Yes
3,1.0,Joe,3.0,3,Graffiti Analysis,Yes
4,5.0,Celeste,3.0,3,Graffiti Analysis,Yes
5,NaN,NaN,NaN,4,Film Studies,No


Notice that the row with the course *Film Studies* does not match any record in the `students` table

### Outer Join (Full Join)

Returns all rows in the `students` and `courses` tables *regardless* of whether the rows in one table matches any row in the other table 

In [9]:
# outer join
outer = pd.merge(students, courses, left_on= 'Course_id', right_on='Id', how='outer')
outer

,Id_x,Name,Course_id,Id_y,Course,Prerequisites
0,4.0,Matt,1.0,1.0,Intro to Harry Potter,No
1,2.0,Maria,2.0,2.0,Intermediate Stats,Yes
2,3.0,Zoe,2.0,2.0,Intermediate Stats,Yes
3,1.0,Joe,3.0,3.0,Graffiti Analysis,Yes
4,5.0,Celeste,3.0,3.0,Graffiti Analysis,Yes
5,NaN,NaN,NaN,4.0,Film Studies,No
6,6.0,Franco,5.0,NaN,NaN,NaN


### Graffiti Database

The most common join you will conducting here is the *inner join*. We can try to understand the differences between these different types of joins by making reference to our Graffiti database:

* **Inner Join**: This refers to an intersection of the two tables. With this type of join we are only going to keep those rows with 'id's in the `canvas` table that match the 'canvas_id's in the `graffiti` table. So, if for instance someone recorded a canvas entry in the `canvas` table but did not record a graffiti instance in the `graffiti` table associated with this canvas entry, thus the entry would not appear after doing the join (indeed this case has occurred several times!). Of course the reverse is also true, if someone entered a new record in the `graffiti` table that did not correspond to a record in the `canvas` table, such record would not appear after doing the join (this is not possible because of how the Graffiti app operates).

* **Left (Outer) Join**: With these join, we would retain all `canvas` records (= rows) independently of whether there is a graffiti record associated with them in the `graffiti` table.

* **Right (Outer) Join**: The reverse of a *left outer join*. In our case this type of join will give us the same as an *inner join* because we cannot have a graffiti entry without first having a canvas entry. However, if we had allowed the entry of graffiti records without the existence of a previous canvas record then the result of this join would have returned all graffiti entries even if these did not have a canvas entry associated with them. Again, not possible in our case!

* **Full (Outer) Join**: The result of this type of join is all the rows that have matching `canvas` ids (i.e. Canvas 'id' == Graffito 'canvas_id') plus other rows that may not have matching `canvas` ids. In our particular case, the results are going to the the same as a *left outer join* . With a full join you get all records of all tables regardless of whether the share a key or not.

#### Read csv files

In [10]:
from pathlib import Path

# creating a relative path to the data folder 
pth = Path('../../data')

In [11]:
# read canvas csv file into a dataframe 
canvas = pd.read_csv(pth / 'canvas.csv')

In [12]:
# read graffito
graffiti = pd.read_csv(pth /'graffiti.csv')

Time to apply what we have learnt to our tables. Let's imagine that we are interested in generating a table with all the graffiti entries that have been found to be at a location with *high* potential visibility?

To produce such a table you first need to combine both with an inner join.

In [13]:
rslt = canvas.merge(graffiti, left_on= 'id', right_on='canvas_id', how='inner')

In [14]:
rslt.head(4)

,id_x,created_at_x,uploaded_at_x,created_by_x,title_x,at_canvas,coords,date_entry_canvas,property_type,property_use,...,technique,marker_type,nip_type,other,num_colors,colors,nature_graffiti,transcribable,message,transcription
0,3,2023-11-27 13:32:35-08:00,2023-11-27 13:40:26-08:00,jsomer@uw.edu,11/27/2023 Wall,Y,"{'latitude': 47.658577, 'longitude': -122.3176...",11/27/2023,comercial,abandoned,...,spray,NaN,NaN,NaN,2,"['black', 'white']","['Image', 'Text']",Y,writter,“Roja”
1,3,2023-11-27 13:32:35-08:00,2023-11-27 13:40:26-08:00,jsomer@uw.edu,11/27/2023 Wall,Y,"{'latitude': 47.658577, 'longitude': -122.3176...",11/27/2023,comercial,abandoned,...,spray,NaN,NaN,NaN,3-5,"['black', 'white', 'red', 'gold']",['Image'],Y,other,Triangle/prism
2,3,2023-11-27 13:32:35-08:00,2023-11-27 13:40:26-08:00,jsomer@uw.edu,11/27/2023 Wall,Y,"{'latitude': 47.658577, 'longitude': -122.3176...",11/27/2023,comercial,abandoned,...,marker,marker,unknown,NaN,1,['white'],['Text'],N,NaN,NaN
3,3,2023-11-27 13:32:35-08:00,2023-11-27 13:40:26-08:00,jsomer@uw.edu,11/27/2023 Wall,Y,"{'latitude': 47.658577, 'longitude': -122.3176...",11/27/2023,comercial,abandoned,...,spray,NaN,NaN,NaN,3-5,"['white', 'black', 'blue']",['Text'],N,NaN,NaN


```{note}
If two tables have columns/Series with the same name, Pandas will add a suffix, here `_x` and `_y`, to each column in order to differentiate them. You can enter your own suffixes. See [`.merge()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html) documentation.
```

Now that we can access location information for each graffiti. We first select the locations with high viewing potential.

In [15]:
sel = rslt.viewing_potential == 'high'

We can now determine what type of graffiti are in such locations

In [16]:
rslt[sel].type.value_counts()

type
tag            1103
sticker         168
throwUp         146
hollow           86
piece            57
other            36
wildstyle        12
pasteUp           6
stencil           4
blockbuster       3
edging            3
Name: count, dtype: int64

Joins allows us to combine information from various tables so that we can build queries that go beyond a single table.

### Additional References

- For some additional information on [joins](https://realpython.com/pandas-merge-join-and-concat/)